<a href="https://colab.research.google.com/github/changsooooooooooo/DL-News-Impact-on-Stock/blob/kaylie/AIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kobert_transformers

In [ ]:
import os
import re
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
from google.colab import drive
from kobert_transformers import *
from transformers import BertModel
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
drive.mount('/content/gdrive/')

###Tokenizing Additional

In [ ]:
batch_size = 16
epoch = 10
max_len = 512

In [ ]:
class TokenMaking():
  def __init__(self, data):
    self.data = data

  def attention_mask(self):
    answer = torch.zeros((batch_size, max_len), dtype = torch.long)
    answer[:, :len(self.data)] = torch.from_numpy(np.random.choice(2, size=(batch_size, len(self.data)), p = [0.8, 0.2]))
    return answer.to("cuda")

  def token_type_id(self):
    return torch.zeros((batch_size, ), dtype=torch.long, device='cuda:0')

In [ ]:
def make_ids(tokenizer, string_):
  tokenized_string = tokenizer.tokenize(string_)
  return tokenizer.convert_tokens_to_ids(tokenized_string)

In [ ]:
os.chdir("./gdrive/MyDrive/AIR")

In [ ]:
data = [pd.read_excel(file) for file in os.listdir() if file.endswith("xlsx")]

In [ ]:
tokenizer = get_tokenizer()

In [ ]:
make_ids(tokenizer, "[CLS] "+data[0].loc[10]["title"]+" [SEP]")

In [ ]:
model = BertModel.from_pretrained("monologg/kobert")

###Simple Preprocessing

In [ ]:
data[0].dropna(subset=["content", "summary"], inplace = True)
data[1].dropna(subset=["content", "summary"], inplace = True)
for i in range(2):
  data[i].loc[:, "content"].replace('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\[\]\<\>`\'…》]', " ", regex=True, inplace=True)
  data[i].loc[:, "content"].replace('\n', "", regex=True, inplace=True)
  data[i].loc[:, "summary"].replace('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\[\]\<\>`\'…》]', " ", regex=True, inplace=True)
  data[i].loc[:, "summary"].replace('\n', "", regex=True, inplace=True)
  data[i].loc[:, "content"].replace('[가-힣]+ 기자', "", regex=True, inplace=True)
  data[i].loc[:, "summary"].replace('[가-힣]+ 기자', "", regex=True, inplace=True)
  data[i].loc[:, "content"].replace('[가-힣]+ 선임기자', "", regex=True, inplace=True)
  data[i].loc[:, "summary"].replace('[가-힣]+ 선임기자', "", regex=True, inplace=True)